In [ ]:
print("Hello")

In [ ]:
!pip install -U accelerate bitsandbytes datasets peft transformers
!pip install huggingface_hub
!pip install -q -U trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from google.colab import drive
import pickle

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig, set_seed
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from trl import SFTTrainer, setup_chat_format
from transformers.trainer_utils import get_last_checkpoint
from accelerate import Accelerator
import logging
import transformers
import datasets
import sys
import os

logger = logging.getLogger(__name__)


# Set seed for reproducibility
set_seed(42)

In [ ]:
# Implemented based on the HuggingFace repository: https://github.com/huggingface/alignment-handbook/blob/main/scripts/run_sft.py

In [ ]:
output_dir = "/content/drive/MyDrive/AI/SycophanticDataGen-v2/"
model_id="mistralai/Mistral-7B-Instruct-v0.2"

In [ ]:
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)
log_level = "INFO"
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

In [ ]:
last_checkpoint = None
if os.path.isdir(output_dir):
    last_checkpoint = get_last_checkpoint(output_dir)

if last_checkpoint is not None:
    logger.info(f"Checkpoint detected, resuming training at {last_checkpoint=}.")


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    revision="main",
    trust_remote_code=False,
    add_eos_token=True,
    use_fast=True
    )
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = 'left'

# Set reasonable default for models without max length
if tokenizer.model_max_length > 100_000:
    tokenizer.model_max_length = 2048

# TODO, I'm not sure, whether I should use chat template for the other dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

[INFO|tokenization_utils_base.py:2108] 2024-05-29 19:13:15,207 >> loading file tokenizer.model from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/tokenizer.model
[INFO|tokenization_utils_base.py:2108] 2024-05-29 19:13:15,209 >> loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/tokenizer.json
[INFO|tokenization_utils_base.py:2108] 2024-05-29 19:13:15,210 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2108] 2024-05-29 19:13:15,211 >> loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/special_tokens_map.json
[INFO|tokenization_utils_base.py:2108] 2024-05-29 19:13:15,212 >> loading file tokenizer_config.json from cache at /root/.c

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_storage="uint8",
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    revision="main",
    trust_remote_code=False,
    # use_flash_attention_2=True,
    torch_dtype=torch.bfloat16,
    use_cache=False,
    device_map="auto",
    # device_map={"": Accelerator().local_process_index},
    quantization_config=quantization_config,
)

# I've skipped
# model, tokenizer = setup_chat_format(model, tokenizer)
# model_kwargs = None

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

[INFO|configuration_utils.py:733] 2024-05-29 19:13:15,512 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/config.json
[INFO|configuration_utils.py:796] 2024-05-29 19:13:15,517 >> Model config MistralConfig {
  "_name_or_path": "mistralai/Mistral-7B-Instruct-v0.2",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.41.1",
  "use_cache": false,
  "vocab_size": 32000
}



model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

[INFO|modeling_utils.py:3474] 2024-05-29 19:13:15,809 >> loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/model.safetensors.index.json


model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

...

[INFO|modeling_utils.py:1519] 2024-05-29 19:15:24,067 >> Instantiating MistralForCausalLM model under default dtype torch.bfloat16.
[INFO|configuration_utils.py:962] 2024-05-29 19:15:24,080 >> Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "use_cache": false
}

[INFO|quantizer_bnb_4bit.py:105] 2024-05-29 19:15:27,128 >> target_dtype {target_dtype} is replaced by `CustomDtype.INT4` for 4-bit BnB quantization


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[INFO|modeling_utils.py:4280] 2024-05-29 19:16:35,058 >> All model checkpoint weights were used when initializing MistralForCausalLM.

[INFO|modeling_utils.py:4288] 2024-05-29 19:16:35,059 >> All the weights of MistralForCausalLM were initialized from the model checkpoint at mistralai/Mistral-7B-Instruct-v0.2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use MistralForCausalLM for predictions without further training.


generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

[INFO|configuration_utils.py:917] 2024-05-29 19:16:35,188 >> loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/generation_config.json
[INFO|configuration_utils.py:962] 2024-05-29 19:16:35,190 >> Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2
}



In [ ]:
model = prepare_model_for_kbit_training(model)
model.config.pad_token_id = tokenizer.pad_token_id

peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
)

In [ ]:
training_arguments = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="steps",
        # do_eval=True,
        optim="paged_adamw_8bit",
        # I guess that's the parameter for A100
        per_device_train_batch_size=12,
        gradient_accumulation_steps=2,
        per_device_eval_batch_size=12,
        # per_device_train_batch_size=4,
        # gradient_accumulation_steps=4,
        # per_device_eval_batch_size=4,
        log_level="debug",
        logging_steps=10,
        learning_rate=4e-4,
        eval_steps=25,
        #num_train_epochs=1,
        max_steps=250,
        save_steps=25,
        warmup_steps=25,
        lr_scheduler_type="linear",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
[INFO|training_args.py:2052] 2024-05-29 19:16:35,543 >> PyTorch: setting up devices
[INFO|training_args.py:1727] 2024-05-29 19:16:35,567 >> The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
NLP_SPLITTER = 'Which label would you assign for the following input'

def clean_from_user_opinion(text):
  return "Human: " + NLP_SPLITTER + text.split(NLP_SPLITTER)[1].strip()

def wrap_human_as_instruct(text):
  split_assistant = text.split('\n\nAssistant:')

  return "[INST]" + split_assistant[0].strip() + "[/INST]\n\nAssistant:" + split_assistant[1].strip()

In [ ]:
with open('/content/drive/MyDrive/AI/nlp_cai_revision_non_sycophantic.pickle', 'rb') as file:
  dataset = pickle.load(file)

with open('/content/drive/MyDrive/AI/nlp_cai_revisions_3_2_100.pickle', 'rb') as file:
  eval_dataset_sycophantic = pickle.load(file)

In [ ]:
from datasets import Dataset
import pandas as pd

dataset_dicts = [{"key": key, "correct_letter": correct_letter, "opinion_letter": opinion_letter, "answer": answer, "revision_answer": revision_answer}
                 for (key, (correct_letter, opinion_letter, answer, critique_statement, critique_answer, revision_statement, revision_answer)) in dataset.items()]

for element_dict in dataset_dicts:
  sycophantic_input = element_dict["key"]
  # This part is present in the revision answer already
  sycophantic_input = sycophantic_input.replace("I believe the best answer is", "")
  revision_answer = element_dict["revision_answer"]
  element_dict["text"] = "<s>" + wrap_human_as_instruct(sycophantic_input) + " " + revision_answer.strip()

sycophantic_dataset_for_training = Dataset.from_pandas(pd.DataFrame(dataset_dicts))

In [ ]:
dataset_dicts = [{"key": key, "correct_letter": correct_letter, "opinion_letter": opinion_letter, "answer": answer, "revision_answer": revision_answer}
                 for (key, (correct_letter, opinion_letter, answer, critique_statement, critique_answer, revision_statement, revision_answer)) in eval_dataset_sycophantic.items()]

for element_dict in dataset_dicts:
  sycophantic_input = element_dict["key"]
  # This part is present in the revision answer already
  sycophantic_input = sycophantic_input.replace("I believe the best answer is", "")
  revision_answer = element_dict["revision_answer"]
  element_dict["text"] = "<s>" + wrap_human_as_instruct(sycophantic_input) + " " + revision_answer.strip()

sycophantic_dataset_for_eval = Dataset.from_pandas(pd.DataFrame(dataset_dicts))

In [ ]:
from datasets import load_dataset, concatenate_datasets

dataset_instruct = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_sft[:1%]")
dataset_instruct = dataset_instruct.shuffle(seed=42).select(range(1000))

dataset_instruct_eval = load_dataset("HuggingFaceH4/ultrachat_200k", split="test_sft[:1%]")
dataset_instruct_eval = dataset_instruct_eval.shuffle(seed=42).select(range(100))

https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k/resolve/f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/README.md not found in cache or force_download set to True, downloading to /root/.cache/huggingface/datasets/downloads/2308d4ab5d4abace4441eb331fb4a84950783ac5fad54bdd2b1bb0e6d874829c.3ecb0462117b7560fd07d072dc04e5ed70d29811052bd142b028753831bc8e72.incomplete
INFO:datasets.utils.file_utils:https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k/resolve/f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/README.md not found in cache or force_download set to True, downloading to /root/.cache/huggingface/datasets/downloads/2308d4ab5d4abace4441eb331fb4a84950783ac5fad54bdd2b1bb0e6d874829c.3ecb0462117b7560fd07d072dc04e5ed70d29811052bd142b028753831bc8e72.incomplete


storing https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k/resolve/f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/README.md in cache at /root/.cache/huggingface/datasets/downloads/2308d4ab5d4abace4441eb331fb4a84950783ac5fad54bdd2b1bb0e6d874829c.3ecb0462117b7560fd07d072dc04e5ed70d29811052bd142b028753831bc8e72
INFO:datasets.utils.file_utils:storing https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k/resolve/f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/README.md in cache at /root/.cache/huggingface/datasets/downloads/2308d4ab5d4abace4441eb331fb4a84950783ac5fad54bdd2b1bb0e6d874829c.3ecb0462117b7560fd07d072dc04e5ed70d29811052bd142b028753831bc8e72
creating metadata file for /root/.cache/huggingface/datasets/downloads/2308d4ab5d4abace4441eb331fb4a84950783ac5fad54bdd2b1bb0e6d874829c.3ecb0462117b7560fd07d072dc04e5ed70d29811052bd142b028753831bc8e72
INFO:datasets.utils.file_utils:creating metadata file for /root/.cache/huggingface/datasets/downloads/2308d4ab5d4abace4441eb331fb4a84950783a

storing hf://datasets/HuggingFaceH4/ultrachat_200k@f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/data/train_sft-00000-of-00003-a3ecf92756993583.parquet in cache at /root/.cache/huggingface/datasets/downloads/2ea1c4b4b741066bce560c9899ac1e726de3810da0926439e2a8a9188efcf481
INFO:datasets.utils.file_utils:storing hf://datasets/HuggingFaceH4/ultrachat_200k@f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/data/train_sft-00000-of-00003-a3ecf92756993583.parquet in cache at /root/.cache/huggingface/datasets/downloads/2ea1c4b4b741066bce560c9899ac1e726de3810da0926439e2a8a9188efcf481
creating metadata file for /root/.cache/huggingface/datasets/downloads/2ea1c4b4b741066bce560c9899ac1e726de3810da0926439e2a8a9188efcf481
INFO:datasets.utils.file_utils:creating metadata file for /root/.cache/huggingface/datasets/downloads/2ea1c4b4b741066bce560c9899ac1e726de3810da0926439e2a8a9188efcf481
hf://datasets/HuggingFaceH4/ultrachat_200k@f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/data/train_sft-00001-of-00003-0a1804bcb6ae68c6

storing hf://datasets/HuggingFaceH4/ultrachat_200k@f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/data/train_sft-00001-of-00003-0a1804bcb6ae68c6.parquet in cache at /root/.cache/huggingface/datasets/downloads/9ea1b34139edf850223cf94ec86dce854653fede94fc6e028153720541d27f3d
INFO:datasets.utils.file_utils:storing hf://datasets/HuggingFaceH4/ultrachat_200k@f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/data/train_sft-00001-of-00003-0a1804bcb6ae68c6.parquet in cache at /root/.cache/huggingface/datasets/downloads/9ea1b34139edf850223cf94ec86dce854653fede94fc6e028153720541d27f3d
creating metadata file for /root/.cache/huggingface/datasets/downloads/9ea1b34139edf850223cf94ec86dce854653fede94fc6e028153720541d27f3d
INFO:datasets.utils.file_utils:creating metadata file for /root/.cache/huggingface/datasets/downloads/9ea1b34139edf850223cf94ec86dce854653fede94fc6e028153720541d27f3d
hf://datasets/HuggingFaceH4/ultrachat_200k@f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/data/train_sft-00002-of-00003-ee46ed25cfae92c6

storing hf://datasets/HuggingFaceH4/ultrachat_200k@f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/data/train_sft-00002-of-00003-ee46ed25cfae92c6.parquet in cache at /root/.cache/huggingface/datasets/downloads/9dcbe7bdea54ea9a4858bd20ce46c02987c2858888facb1b9a5955586b73dc2b
INFO:datasets.utils.file_utils:storing hf://datasets/HuggingFaceH4/ultrachat_200k@f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/data/train_sft-00002-of-00003-ee46ed25cfae92c6.parquet in cache at /root/.cache/huggingface/datasets/downloads/9dcbe7bdea54ea9a4858bd20ce46c02987c2858888facb1b9a5955586b73dc2b
creating metadata file for /root/.cache/huggingface/datasets/downloads/9dcbe7bdea54ea9a4858bd20ce46c02987c2858888facb1b9a5955586b73dc2b
INFO:datasets.utils.file_utils:creating metadata file for /root/.cache/huggingface/datasets/downloads/9dcbe7bdea54ea9a4858bd20ce46c02987c2858888facb1b9a5955586b73dc2b
hf://datasets/HuggingFaceH4/ultrachat_200k@f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/data/test_sft-00000-of-00001-f7dfac4afe5b93f4.

...

Generating train_gen split
INFO:datasets.builder:Generating train_gen split


Generating train_gen split:   0%|          | 0/256032 [00:00<?, ? examples/s]

Generating test_gen split
INFO:datasets.builder:Generating test_gen split


Generating test_gen split:   0%|          | 0/28304 [00:00<?, ? examples/s]

All the splits matched successfully.
INFO:datasets.utils.info_utils:All the splits matched successfully.
Dataset ultrachat_200k downloaded and prepared to /root/.cache/huggingface/datasets/HuggingFaceH4___ultrachat_200k/default/0.0.0/f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb. Subsequent calls will reuse this data.
INFO:datasets.builder:Dataset ultrachat_200k downloaded and prepared to /root/.cache/huggingface/datasets/HuggingFaceH4___ultrachat_200k/default/0.0.0/f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb. Subsequent calls will reuse this data.
Caching indices mapping at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrachat_200k/default/0.0.0/f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/cache-d9e123625e167c34.arrow
INFO:datasets.arrow_dataset:Caching indices mapping at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrachat_200k/default/0.0.0/f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/cache-d9e123625e167c34.arrow
Overwrite dataset info from restored data version if exists.
INFO:dataset

In [ ]:
def apply_chat_template(
    example,
    tokenizer,
    auto_insert_empty_system_msg: bool = True,
):
  messages = example["messages"]

  example["text"] = tokenizer.apply_chat_template(
      messages,
      tokenize=False
  )

  return example


In [ ]:
dataset_instruct_prepared = dataset_instruct.map(apply_chat_template,
        fn_kwargs={
            "tokenizer": tokenizer
        },
        remove_columns=list(dataset_instruct.features),
        desc="Applying chat template",
)

dataset_instruct_eval_prepared = dataset_instruct_eval.map(apply_chat_template,
        fn_kwargs={
            "tokenizer": tokenizer
        },
        remove_columns=list(dataset_instruct_eval.features),
        desc="Applying chat template",
)

Applying chat template:   0%|          | 0/1000 [00:00<?, ? examples/s]

Caching processed dataset at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrachat_200k/default/0.0.0/f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/cache-b0ca7bdd3811220b.arrow
INFO:datasets.arrow_dataset:Caching processed dataset at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrachat_200k/default/0.0.0/f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/cache-b0ca7bdd3811220b.arrow


Applying chat template:   0%|          | 0/100 [00:00<?, ? examples/s]

Caching processed dataset at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrachat_200k/default/0.0.0/f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/cache-004906f0eefeb133.arrow
INFO:datasets.arrow_dataset:Caching processed dataset at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrachat_200k/default/0.0.0/f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/cache-004906f0eefeb133.arrow


In [ ]:
dataset_for_training = concatenate_datasets([dataset_instruct_prepared, sycophantic_dataset_for_training]).shuffle(seed=42)
dataset_for_eval = concatenate_datasets([dataset_instruct_eval_prepared, sycophantic_dataset_for_eval]).shuffle(seed=42).select(range(24))

Some of the datasets have disparate format. Resetting the format of the concatenated dataset.
INFO:datasets.arrow_dataset:Some of the datasets have disparate format. Resetting the format of the concatenated dataset.
Caching indices mapping at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrachat_200k/default/0.0.0/f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/cache-586db86b8e25bd31.arrow
INFO:datasets.arrow_dataset:Caching indices mapping at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrachat_200k/default/0.0.0/f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/cache-586db86b8e25bd31.arrow
Some of the datasets have disparate format. Resetting the format of the concatenated dataset.
INFO:datasets.arrow_dataset:Some of the datasets have disparate format. Resetting the format of the concatenated dataset.
Caching indices mapping at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrachat_200k/default/0.0.0/f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/cache-ffdbf3c522d5945f.arrow
INFO:dataset

In [ ]:
dataset_for_training.to_pandas()

,text,key,correct_letter,opinion_letter,answer,revision_answer
0,<s>[INST] Write a fictional short story about ...,None,None,None,None,None
1,<s>[INST] Imagine a conversation between two c...,None,None,None,None,None
2,"<s>[INST]Human: Hello, my name is Emily Young....","Human: Hello, my name is Emily Young. I am a 5...",(B),(A),(B) Neither Entailment Nor Contradiction. Th...,I believe the best answer is (B) Neither Enta...
3,"<s>[INST]Human: Hello, my name is Terri Powers...","Human: Hello, my name is Terri Powers. I am a ...",(A),(A),(A) Not Entailment. The first statement disc...,I believe the best answer is (A) Not Entailme...
4,<s>[INST] Can you provide a list of the file f...,None,None,None,None,None
...,...,...,...,...,...,...
1774,<s>[INST] The function should then return the ...,None,None,None,None,None
1775,"<s>[INST]Human: Hello, my name is William Lee....","Human: Hello, my name is William Lee. I am a 8...",(A),(B),(A) Neither Entailment Nor Contradiction. Th...,I believe the best answer is (A) Neither Enta...
1776,"<s>[INST]Human: Hello, my name is Samuel Alexa...","Human: Hello, my name is Samuel Alexander. I a...",(A),(A),(A) Neither Entailment Nor Contradiction. Th...,I believe the best answer is (A) Neither Enta...
1777,<s>[INST] Given the text: Tennessee Gov. Bill ...,None,None,None,None,None


In [ ]:
trainer = SFTTrainer(
        model=model,
        train_dataset=dataset_for_training,
        eval_dataset=dataset_for_eval,
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=training_arguments,
)

trainer.train()

# So, it looks like, I will process ~400 examples during this training, it will take roughly ~2:20
# I could project that SFT for the CAI case will likely require ~1500 examples (+1500 Instruct Examples), processing of these examples will likely take ~19 hours
# Once it is done, my next plan will be to generate another 1500 examples and set up RLHF on them. I could take the new data or stay on the same data as well.


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
[INFO|training_args.py:2052] 2024-05-29 19:17:47,395 >> PyTorch: setting up devices


Map:   0%|          | 0/1779 [00:00<?, ? examples/s]

Caching processed dataset at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrachat_200k/default/0.0.0/f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/cache-b9bb1b914942c51a.arrow
INFO:datasets.arrow_dataset:Caching processed dataset at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrachat_200k/default/0.0.0/f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/cache-b9bb1b914942c51a.arrow


Map:   0%|          | 0/24 [00:00<?, ? examples/s]

Caching processed dataset at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrachat_200k/default/0.0.0/f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/cache-ac86dbf01d310911.arrow
INFO:datasets.arrow_dataset:Caching processed dataset at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrachat_200k/default/0.0.0/f8e46c0ce6e7cfa42c393cb56add1db4ea9548fb/cache-ac86dbf01d310911.arrow
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
[WARNING|trainer.py:591] 2024-05-29 19:17:51,856 >> max_steps is given, it will override any value given in num_train_epochs
[DEBUG|trainer.py:1912] 2024-05-29 19:17:52,188 >> Currently training with a batch size of: 12
[INFO|trainer.py:2078] 2024

Step,Training Loss,Validation Loss
25,1.287600,0.939900
50,1.059200,0.863258
75,1.033200,0.850304
100,0.836000,0.875026
125,0.846000,0.876274
150,0.845500,0.878215
175,0.584700,0.969726
200,0.577900,0.974241
225,0.541700,0.977780
250,0.368200,1.081329


[INFO|trainer.py:3719] 2024-05-29 20:24:16,831 >> ***** Running Evaluation *****
[INFO|trainer.py:3721] 2024-05-29 20:24:16,835 >>   Num examples = 24
[INFO|trainer.py:3724] 2024-05-29 20:24:16,836 >>   Batch size = 12
[INFO|trainer.py:3410] 2024-05-29 20:25:05,556 >> Saving model checkpoint to /content/drive/MyDrive/AI/SycophanticDataGen-v2/checkpoint-25
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
[INFO|configuration_utils.py:733] 2024-05-29 20:25:05,794 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/config.json
[INFO|configuration_utils.py:796] 2024-05-29 20:25:05,796 >> Model config MistralConfig {
  "architectures":

TrainOutput(global_step=250, training_loss=0.8208622121810913, metrics={'train_runtime': 40165.644, 'train_samples_per_second': 0.149, 'train_steps_per_second': 0.006, 'total_flos': 1.3124351548823962e+17, 'train_loss': 0.8208622121810913, 'epoch': 3.3557046979865772})

In [ ]:
from transformers import GenerationConfig
from peft import PeftModel
model.config.use_cache = True
model = PeftModel.from_pretrained(model, output_dir+'checkpoint-250/')
def generate(instruction):
    prompt = "### Human: "+instruction+"### Assistant: "
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
            input_ids=input_ids,
            generation_config=GenerationConfig(pad_token_id=tokenizer.pad_token_id, temperature=1.0, top_p=1.0, top_k=50, num_beams=1),
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=256
    )
    for seq in generation_output.sequences:
        output = tokenizer.decode(seq)
        print(output.split("### Assistant: ")[1].strip())
generate("Tell me about gravitation.")

</s> [/INST]Gravitation is the fundamental force that attracts two bodies towards each other. It is the force that keeps planets in orbit around the sun, and moons in orbit around their planets. The strength of the gravitational force depends on the mass of the two objects and the distance between them.

Gravitation was first described by Sir Isaac Newton in his law of universal gravitation. According to this law, every point mass attracts every other point mass by a force that is directly proportional to the product of their masses and inversely proportional to the square of the distance between their centers.

Later, Albert Einstein described gravitation in his theory of general relativity. According to this theory, gravity is not a force, but a result of the curvature of space-time caused by the presence of mass and energy.

Gravitation plays a crucial role in our understanding of the universe. It helps us understand the motion of celestial bodies, the structure of galaxies, and the

In [ ]:
# from google.colab import runtime
# runtime.unassign()
